In [1]:
# -- Run this cell to install the required packages
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install folium
%pip install geopandas
%pip install shapely
%pip install networkx
%pip install osmnx
%pip install gurobipy


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import re
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import colormaps as cm

import folium
from folium import plugins
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point, LineString, box
import networkx as nx
import gurobipy as gb

# Project setup

### Load data

In [3]:
# Get segment and stops on route
def get_segment_stops(
    segment_df: pd.DataFrame,
    stops_df: pd.DataFrame,
    route_id: str,
    direction_id: int = 0,
):
    segment_distance = segment_df.query(
        "route_id == @route_id & direction_id == @direction_id"
    )

    stops_in_route = stops_df[
        stops_df["stop_id"].isin(
            list(
                set(
                    segment_distance[["start_stop_id", "end_stop_id"]].values.reshape(
                        -1
                    )
                )
            )
        )
    ]

    return segment_distance, stops_in_route


# Get unique stops on route
def get_stops_on_route(
    route_ids: list, segment_df: pd.DataFrame, stops_df: pd.DataFrame
):
    route_details = {}

    for r in route_ids:
        detail = get_segment_stops(segment_df, stops_df, r, 0)

        route_details[r] = {}
        route_details[r]["segment"] = detail[0]
        route_details[r]["stops"] = detail[1]

    stops_df = pd.concat(
        [v["stops"] for k, v in route_details.items()]
    ).drop_duplicates()

    print(f"Number of total stops in routes: {len(stops_df)}")

    stops_df["geometry"] = stops_df.apply(
        lambda x: Point((float(x.stop_lon), float(x.stop_lat))), axis=1
    )

    stops_df_gpd = gpd.GeoDataFrame(
        stops_df.drop(
            columns=["location_type", "parent_station", "wheelchair_boarding"]
        ),
        geometry="geometry",
    )

    display(stops_df_gpd.head())

    return stops_df_gpd

In [4]:
segments = pd.read_csv("segments.csv")
stops = pd.read_csv("stops.csv")

segments[["route_id", "start_stop_id", "end_stop_id"]] = segments[
    ["route_id", "start_stop_id", "end_stop_id"]
].astype(str)

routes = ["24", "51", "67", "18", "33", "45", "80"]

stops_df_gpd = get_stops_on_route(routes, segments, stops)

Number of total stops in routes: 326


,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,geometry
1769,51241,51241,Station Villa-Maria,45.479704,-73.619643,https://www.stm.info/fr/recherche#stq=51241,POINT (-73.61964 45.47970)
1806,51281,51281,Décarie / Duquette,45.478389,-73.618050,https://www.stm.info/fr/recherche#stq=51281,POINT (-73.61805 45.47839)
1848,51326,51326,Décarie / Notre-Dame-de-Grâce,45.477244,-73.615513,https://www.stm.info/fr/recherche#stq=51326,POINT (-73.61551 45.47724)
1891,51372,51372,Décarie / Côte-Saint-Antoine,45.476312,-73.613443,https://www.stm.info/fr/recherche#stq=51372,POINT (-73.61344 45.47631)
1976,51462,51462,Décarie / Sherbrooke,45.474535,-73.609490,https://www.stm.info/fr/recherche#stq=51462,POINT (-73.60949 45.47454)


#### Add depot

In [5]:
def add_depot(lat: float, lon: float, stops_df: pd.DataFrame):
    depot = pd.DataFrame(
        {
            "stop_id": ["0"],
            "stop_name": ["Depot"],
            "stop_lat": [lat],
            "stop_lon": [lon],
            "stop_code": 0.0,
        }
    )

    depot["geometry"] = Point((float(depot.stop_lon), float(depot.stop_lat)))

    stops_df_gpd = pd.concat([depot, stops_df]).reset_index(drop=True)

    return depot, stops_df_gpd

### Take a random sample of stops

In [6]:
NUM_STOPS = 25
random_stops_df_gpd = stops_df_gpd.sample(n=NUM_STOPS, random_state=5)
depot, random_stops_df_gpd = add_depot(45.509642, -73.580091, random_stops_df_gpd)

print(f"Number of stops in sample: {len(random_stops_df_gpd)}")

display(random_stops_df_gpd.head())

Number of stops in sample: 26


,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,stop_url
0,0,Depot,45.509642,-73.580091,0.0,POINT (-73.580091 45.509642),NaN
1,52718,De Lorimier / Sherbrooke,45.530713,-73.562790,52718.0,POINT (-73.56279 45.530713),https://www.stm.info/fr/recherche#stq=52718
2,54638,du Val d'Anjou / de la Nantaise,45.597701,-73.553117,54638.0,POINT (-73.553117 45.597701),https://www.stm.info/fr/recherche#stq=54638
3,51511,Beaubien / Saint-Denis,45.533853,-73.605056,51511.0,POINT (-73.605056 45.533853),https://www.stm.info/fr/recherche#stq=51511
4,50516,Papineau / Sauriol,45.566957,-73.650301,50516.0,POINT (-73.650301 45.566957),https://www.stm.info/fr/recherche#stq=50516


## Calculate distance matrix

In [7]:
G = ox.load_graphml("montreal_drive.graphml")

def get_distance_matrix(stops_df: pd.DataFrame, G: nx.Graph):
    if os.path.exists("distance_matrix.json"):
        print("Loading distance matrix from JSON")
        distance_matrix = pd.read_json("distance_matrix.json")

        distance_matrix.columns = distance_matrix.columns.astype(str)
        distance_matrix.index = distance_matrix.index.astype(str)

        if len(distance_matrix) != len(stops_df):
            print("⚠️ Distance matrix does not match number of stops")
            print(f"Number of stops in distance matrix: {len(distance_matrix)}")
            print(f"Number of stops in DataFrame: {len(stops_df)}")
        else:
            print("✅ Distance matrix loaded successfully")
            print(f"Number of stops in distance matrix: {len(distance_matrix)}")
            print(f"Number of stops in DataFrame: {len(stops_df)}")

        return distance_matrix

    print(f"Calculating distance for {len(stops_df)} stops")

    distance_matrix = np.zeros((len(stops_df), len(stops_df)))

    for i, stop1 in enumerate(stops_df.itertuples()):
        print(f"Calculating distance for stop {i}")
        for j in range(i + 1, len(stops_df)):
            stop2 = stops_df.iloc[j]

            origin = ox.nearest_nodes(G, stop1.stop_lon, stop1.stop_lat)
            destination = ox.nearest_nodes(G, stop2.stop_lon, stop2.stop_lat)

            try:
                distance = nx.shortest_path_length(
                    G, origin, destination, weight="length"
                )
            except nx.NetworkXNoPath:
                distance = np.Inf

            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance

        print("-" * 50)

    # Convert to km and save as JSON
    distance_matrix = pd.DataFrame(
        distance_matrix / 1000,
        columns=stops_df.stop_id,
        index=stops_df.stop_id,
    )

    distance_matrix.to_json("distance_matrix.json")

    return distance_matrix

distance_matrix = get_distance_matrix(random_stops_df_gpd, G)

Calculating distance for 26 stops
Calculating distance for stop 0
--------------------------------------------------
Calculating distance for stop 1
--------------------------------------------------
Calculating distance for stop 2
--------------------------------------------------
Calculating distance for stop 3
--------------------------------------------------
Calculating distance for stop 4
--------------------------------------------------
Calculating distance for stop 5
--------------------------------------------------
Calculating distance for stop 6
--------------------------------------------------
Calculating distance for stop 7
--------------------------------------------------
Calculating distance for stop 8
--------------------------------------------------
Calculating distance for stop 9
--------------------------------------------------
Calculating distance for stop 10
--------------------------------------------------
Calculating distance for stop 11
-------------------

## Add disaster area

In [8]:
def add_disaster_area(stops_df: pd.DataFrame, disaster_bounds: list):
    disaster_area = box(*disaster_bounds)

    stops_df_gpd = gpd.GeoDataFrame(stops_df, geometry="geometry")

    stops_in_disaster_area = stops_df_gpd[stops_df_gpd.within(disaster_area)]

    print(f"Number of stops in disaster area: {len(stops_in_disaster_area)}")

    display(stops_in_disaster_area)

    return stops_in_disaster_area, disaster_area


stops_in_disaster_area, disaster_area = add_disaster_area(
    random_stops_df_gpd, [-73.666166, 45.547247, -73.591552, 45.637577]
)

Number of stops in disaster area: 5


,stop_id,stop_name,stop_lat,stop_lon,stop_code,geometry,stop_url
4,50516,Papineau / Sauriol,45.566957,-73.650301,50516.0,POINT (-73.65030 45.56696),https://www.stm.info/fr/recherche#stq=50516
7,55333,Saint-Michel / Henri-Bourassa,45.583717,-73.649799,55333.0,POINT (-73.64980 45.58372),https://www.stm.info/fr/recherche#stq=55333
12,55278,Henri-Bourassa / Désy,45.607374,-73.617516,55278.0,POINT (-73.61752 45.60737),https://www.stm.info/fr/recherche#stq=55278
23,55057,Saint-Michel / Fleury,45.579543,-73.643171,55057.0,POINT (-73.64317 45.57954),https://www.stm.info/fr/recherche#stq=55057
24,51469,Saint-Michel / Crémazie,45.562920,-73.606652,51469.0,POINT (-73.60665 45.56292),https://www.stm.info/fr/recherche#stq=51469


## View stops in sample

In [9]:
stops_map = folium.Map(
    location=[45.5048542, -73.5691235],
    zoom_start=11,
    tiles="cartodbpositron",
    width="100%",
)

# Add disaster area
folium.GeoJson(
    disaster_area,
    name="Disaster area",
    style_function=lambda x: {
        "color": "#ff0000",
        "fillColor": "#ff0000",
        "weight": 1,
        "fillOpacity": 0.4,
    },
).add_to(stops_map)

# Add stops in network
for stop in random_stops_df_gpd.itertuples():
    folium.CircleMarker(
        location=[stop.stop_lat, stop.stop_lon],
        radius=5,
        color=(
            "red"
            if stop.stop_id in stops_in_disaster_area.stop_id.values.tolist()
            else "darkgreen"
        ),
        fill=True,
        fill_opacity=1,
        fill_color=(
            "red"
            if stop.stop_id in stops_in_disaster_area.stop_id.values.tolist()
            else "darkgreen"
        ),
        tooltip=f"{stop.stop_name} ({stop.stop_id})",
        popup=f"""
        <div>
            <h4>{stop.stop_name} ({stop.stop_id})</h4>
            <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
        </div>
        """,
    ).add_to(stops_map)

folium.plugins.Fullscreen(position="topright").add_to(stops_map)
folium.plugins.MousePosition(position="topright").add_to(stops_map)

stops_map

# MILP Model - Split delivery vehicle routing problem

## Split nodes

### Split methods

#### Geometric progression

$$
\begin{align*}
D_{ix} &= \frac{2^{x-1}}{\sum_{i=1}^{S} 2^{x-1}} D_{i}
\end{align*}
$$

$D_{ix}$ is rounded down to the nearest integer. If $\sum_{i=1}^{S} D_{ix} < D_{i}$, then $D_{iS} = D_{iS} + D_{i} - \sum_{i=1}^{S} D_{ix}$.

In [10]:
# Split the node demand into smaller demands according to a geometric progression
def split_demand_geometric(node_demand, BUS_CAPACITY, fraction, fraction_sum):
    demands = [np.floor(node_demand * (f / fraction_sum)) for f in fraction]
    demands = list(filter(lambda x: x > 0, demands))

    if sum(demands) < node_demand:
        demands[-1] += node_demand - sum(demands)

    i = len(demands) - 1

    while demands[i] >= BUS_CAPACITY * 1:
        new_demand = split_demand_geometric(
            demands[i], BUS_CAPACITY, fraction, fraction_sum
        )
        demands.pop(i)
        demands.extend(new_demand)
        i -= 1

    return demands

#### Capacity-based split

$$
\begin{align*}
D_{ix} &= Q &\quad \forall x \in \{1, \dots, S = \frac{q_{i}}{Q}\} \\
\end{align*}
$$

$D_{ix}$ is rounded down to the nearest integer. If $\sum_{i=1}^{S} D_{ix} < D_{i}$, then $D_{i(S+1)} = D_{i} - \sum_{i=1}^{S} D_{ix}$.


In [11]:
# Split the node demand into smaller demands according to BUS_CAPACITY
def split_demand_capacity(node_demand, BUS_CAPACITY):
    demands = [BUS_CAPACITY for _ in range(int(np.floor(node_demand / BUS_CAPACITY)))]
    demands.append(node_demand - sum(demands))

    return demands

#### Equal split

$$
\begin{align*}
D_{ix} &= 1 &\quad \forall x \in \{1, \dots, q_{i}\} \\
\end{align*}
$$


In [12]:
# Split the node demand into equal demand nodes of 1
def split_demand_equal(node_demand, BUS_CAPACITY):
    demands = [1 for _ in range(node_demand)]

    return demands

#### Random split

$$
\begin{align*}
D_{ix} &= Z \sim \{1, Q\}
\end{align*}
$$


In [13]:
# Split the node demand into random demand values

def split_demand_random(node_demand, BUS_CAPACITY, rng: np.random.Generator):
    demands = []
    remaining_demand = node_demand

    while remaining_demand > 0:
        demand = rng.integers(1, BUS_CAPACITY)
        demands.append(demand)
        remaining_demand -= demand

    if sum(demands) > node_demand:
        demands[-1] -= sum(demands) - node_demand

    return demands

#### Split individual demand nodes

In [14]:
def split_demand_node(demand, node, BUS_CAPACITY, rng, split_type: str = "geometric"):
    node_demand = demand[node]
    match split_type:
        case "geometric":
            S = 100
            fraction = [2 ** (i - 1) for i in range(1, S + 1)]
            fraction_sum = sum(fraction)
            demands = split_demand_geometric(
                node_demand, BUS_CAPACITY, fraction, fraction_sum
            )
        case "capacity":
            demands = split_demand_capacity(node_demand, BUS_CAPACITY)
        case "equal":
            demands = split_demand_equal(node_demand, BUS_CAPACITY)
        case "random":
            demands = split_demand_random(node_demand, BUS_CAPACITY, rng)
        case _:
            raise ValueError("Invalid demand split type")

    return {f"{node}_{i}": d for i, d in enumerate(demands, 1)}

### Reconstruct parameters

In [15]:
def reconstruct_demand(demand, BUS_CAPACITY, rng, split_type: str = "geometric"):
    nodes_exceeding_demand = {}
    new_demand = demand.copy()
    for k, v in demand.items():
        if v > BUS_CAPACITY:
            new_nodes = split_demand_node(demand, k, BUS_CAPACITY, rng, split_type)

            new_demand.pop(k)
            new_demand.update(new_nodes)

            nodes_exceeding_demand[k] = new_nodes

    print(f"New nodes added for: {list(nodes_exceeding_demand.keys())}")

    return new_demand, nodes_exceeding_demand

In [16]:
# Update distance matrix to include split nodes. Distance between split nodes of same parent node is 0
def update_distance_matrix(distance_matrix, nodes_exceeding_demand, new_demand):
    distance_matrix_model = distance_matrix.copy()

    for node in nodes_exceeding_demand:
        for i in range(1, len([k for k in new_demand.keys() if node in k]) + 1):
            distance_matrix_model[f"{node}_{i}"] = distance_matrix_model[node]
            distance_matrix_model.loc[f"{node}_{i}"] = distance_matrix_model.loc[node]
            distance_matrix_model.loc[f"{node}_{i}", node] = 0

        distance_matrix_model.drop(columns=[node], index=[node], inplace=True)

    return distance_matrix_model

## Model

In [17]:
def define_model(
    stops: list,
    buses: int,
    demand: dict,
    distance_matrix: pd.DataFrame,
    BUS_CAPACITY: int,
    DISTANCE_THRESHOLD: float = 5.0,
    **params,
):
    # ----------------------------------------------------------------------------------------------
    # Model

    model = gb.Model("Bus Routing")
    model.Params.MIPGap = params.get("MIPGap", 0.05)
    model.Params.TimeLimit = params.get("TimeLimit", 60 * 3)
    model.Params.MIPFocus = params.get("MIPFocus", 1)
    model.Params.LogToConsole = params.get("LogToConsole", 1)

    # ----------------------------------------------------------------------------------------------
    # Decision Variables

    x = model.addVars(
        stops,
        stops,
        buses,
        vtype=gb.GRB.BINARY,
        name=(
            f"{i} -> {j} (bus {k})" for i in stops for j in stops for k in range(buses)
        ),
    )

    u = model.addVars(
        stops,
        vtype=gb.GRB.INTEGER,
        name=(f"Load at Stop {i}" for i in stops),
    )

    # ----------------------------------------------------------------------------------------------
    # Objective Function
    model.setObjective(
        gb.quicksum(
            distance_matrix.loc[i, j] * x[i, j, k]
            for i in stops
            for j in stops
            for k in range(buses)
        ),
        gb.GRB.MINIMIZE,
    )

    # ----------------------------------------------------------------------------------------------
    # Constraints

    # Vehicle leaves nodes that it enters
    model.addConstrs(
        (
            gb.quicksum(x[j, i, k] for j in stops)
            == gb.quicksum(x[i, j, k] for j in stops)
            for i in stops
            for k in range(buses)
        ),
        name="Vehicle leaves nodes that it enters",
    )

    # Every node is entered once
    model.addConstrs(
        (
            gb.quicksum(x[i, j, k] for i in stops for k in range(buses)) == 1
            for j in stops[1:]
        ),
        name="Every node is entered once",
    )

    # Every vehicle leaves the depot
    model.addConstrs(
        (gb.quicksum(x[stops[0], j, k] for j in stops[1:]) <= 1 for k in range(buses)),
        name="Every vehicle may leave the depot if needed",
    )

    # Capacity constraint
    model.addConstrs(
        (
            gb.quicksum(demand[j] * x[i, j, k] for j in stops[1:] for i in stops)
            <= BUS_CAPACITY
            for k in range(buses)
        ),
        name="Capacity constraint",
    )

    # No travel between same node
    model.addConstrs(
        (x[i, i, k] == 0 for i in stops for k in range(buses)),
        name="No same node",
    )

    # Subtour elimination constraints
    model.addConstrs(
        (
            u[j] - u[i] >= demand[j] - BUS_CAPACITY * (1 - x[i, j, k])
            for i in stops[1:]
            for j in stops[1:]
            for k in range(buses)
            if i != j
        ),
        name="Subtour elimination constraint",
    )

    model.addConstrs(
        (u[i] >= demand[i] for i in stops[1:]),
        name="Lower bound for u",
    )

    model.addConstrs(
        (u[i] <= BUS_CAPACITY for i in stops[1:]),
        name="Upper bound for u",
    )

    # Distance between two travel nodes is less than specified distance
    model.addConstrs(
        (
            distance_matrix.loc[i, j] * x[i, j, k] <= DISTANCE_THRESHOLD
            for i in stops[1:]
            for j in stops[1:]
            for k in range(buses)
        ),
        name="Distance between two travel nodes is less than a specified distance",
    )

    # ----------------------------------------------------------------------------------------------
    # Solve model
    model._vars = x
    model.update()

    return model

## Solution

### Optimal routes

In [18]:
def show_solution(model, stops, buses, demand, nodes_exceeding_demand, distance_matrix, stops_df):
    x = model._vars

    print('-'*100)
    print(f"Objective value: {model.objVal:.2f} km")
    print('-'*100)

    # Distance by bus
    distance_bus = pd.DataFrame(
        {
            "bus": k,
            "distance": sum(
                distance_matrix.loc[i, j] * x[i, j, k].x for i in stops for j in stops
            ),
        }
        for k in range(buses)
    )

    distance_bus = distance_bus[distance_bus["distance"] > 0].reset_index(drop=True)

    # display(distance_bus)

    # Bus paths
    bus_path = {}

    for k in range(buses):
        bus_path[k] = []
        for i in stops:
            for j in stops:
                if x[i, j, k].x == 1:
                    bus_path[k].append(
                        {
                            "start_stop": i,
                            "end_stop": j,
                            "distance": distance_matrix.loc[i, j],
                        }
                    )

    # Convert to dataframe with bus route number
    bus_path_df = []

    for k, v in bus_path.items():
        bus_path_df.append(pd.DataFrame(v).assign(bus=k))

    bus_path_df = pd.concat(bus_path_df)

    paths = {}

    grouped = bus_path_df.groupby("bus")

    # Iterate over each bus group
    for bus, group in grouped:
        sorted_group = group.sort_values(by=["start_stop", "end_stop"]).reset_index(
            drop=True
        )
        path = ["0"]  # Initialize the path with the depot

        # Start with the first stop after the depot
        current_stop = sorted_group.loc[
            sorted_group["start_stop"] == "0", "end_stop"
        ].values[0]
        path.append(current_stop)

        # Follow the chain of stops
        while True:
            # Find the next stop where the current stop is the start stop
            next_stop = sorted_group.loc[
                sorted_group["start_stop"] == current_stop, "end_stop"
            ].values
            if not next_stop:
                break  # If there is no next stop, we've completed the path
            next_stop = next_stop[0]

            # Add the next stop to the path and set it as the current stop
            if next_stop == "0":
                break  # If the next stop is the depot, we've completed the path
            path.append(next_stop)
            current_stop = next_stop

        # Store the path for this bus
        paths[bus] = path

    bus_path_df["step"] = bus_path_df.apply(
        lambda x: paths[x.bus].index(x.start_stop), axis=1
    )

    bus_path_df.sort_values(by=["bus", "step"], inplace=True)

    bus_path_df["demand"] = bus_path_df["end_stop"].map(demand)

    bus_path_df[["start_stop", "end_stop"]] = bus_path_df[
        ["start_stop", "end_stop"]
    ].applymap(lambda x: x.split("_")[0])

    bus_path_df = bus_path_df.merge(
        stops_df[["stop_id", "stop_name", "geometry"]],
        left_on="end_stop",
        right_on="stop_id",
        how="left",
    )

    bus_path_df["step_demand"] = bus_path_df.groupby("bus")["demand"].cumsum()

    bus_path_df["is_split"] = bus_path_df["stop_id"].isin(nodes_exceeding_demand.keys())

    num_buses_used = bus_path_df.bus.nunique()
    print(f"Number of buses used: {num_buses_used}")

    # display(bus_path_df)

    # Print routes
    # for route in paths:
    #     print(f"Bus {route}: {' -> '.join(paths[route])} -> 0")

    return distance_bus, bus_path_df, paths

In [19]:
def plot_networkx_bus(bus_path_df):
    plots = {}
    for bus in bus_path_df.bus.unique():
        # print(f"Bus {bus + 1}:")

        bus_route = bus_path_df[bus_path_df["bus"] == bus].reset_index(drop=True)

        fig = plt.figure(figsize=(20, 10))

        g = nx.DiGraph()

        for segment in bus_route.itertuples():
            g.add_edge(
                segment.start_stop,
                segment.end_stop,
                weight=segment.distance,
                step=segment.step,
                demands={"demand": segment.demand, "load": segment.step_demand},
                label=f"{segment.start_stop}-{segment.end_stop}",
            )

        pos = nx.circular_layout(g)

        nx.draw_networkx(g, pos, with_labels=True, node_size=500, node_color="skyblue")
        nx.draw_networkx_edge_labels(
            g, pos, edge_labels=nx.get_edge_attributes(g, "demands")
        )

        plots[bus] = fig

    plt.close("all")
    return plots


# network_plots = plot_networkx_bus(bus_path_df)

In [20]:
def build_route_df(bus_path_df, depot):
    routes_gdf = (
        gpd.GeoDataFrame(bus_path_df.groupby("bus")["geometry"].apply(list))
        .rename(columns={"points": "geometry"})
        .reset_index()
    )

    # add depot to each geometry
    routes_gdf["points"] = routes_gdf.apply(
        lambda x: [Point((float(depot.stop_lon), float(depot.stop_lat)))] + x.geometry,
        axis=1,
    )

    routes_gdf["geometry"] = routes_gdf["points"].apply(LineString)
    routes_gdf.drop(columns=["points"], inplace=True)

    routes_gdf = routes_gdf.merge(
        bus_path_df.groupby("bus")["demand"].sum().rename("demand").reset_index(),
        on="bus",
    )

    routes_gdf = gpd.GeoDataFrame(routes_gdf, geometry="geometry")
    routes_gdf.crs = "EPSG:4326"

    # display(routes_gdf)

    return routes_gdf

### Map the routes

In [21]:
def plot_routes(distance_matrix, bus_path_df, routes_gdf, num_buses, split_type):
    colormap_route = [
        mcolors.rgb2hex(c) for c in list(plt.cm.rainbow(np.linspace(0, 1, num_buses)))
    ]

    route_map = folium.Map(
        location=[45.5048542, -73.5691235],
        zoom_start=11,
        tiles="cartodbpositron",
        width="100%",
    )

    folium.GeoJson(
        disaster_area,
        name="Disaster area",
        style_function=lambda x: {
            "color": "#ff0000",
            "fillColor": "#ff0000",
            "weight": 1,
            "fillOpacity": 0.3,
        },
    ).add_to(route_map)

    mc = folium.plugins.MarkerCluster(name="Individual stops").add_to(route_map)

    for stop in bus_path_df.itertuples():
        c = folium.CircleMarker(
            location=[stop.geometry.coords[0][1], stop.geometry.coords[0][0]],
            radius=5,
            color=(
                colormap_route[stop.bus]
                if not stop.is_split
                else "purple"
                if stop.stop_id != "0"
                else "black"
            ),
            fill=True,
            fill_opacity=1,
            fill_color=(
                colormap_route[stop.bus]
                if not stop.is_split
                else "purple"
                if stop.stop_id != "0"
                else "black"
            ),
            tooltip=f"""
            <b>{stop.stop_name} ({stop.stop_id})</b>
            <br>
            Route: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['bus'].values[0] + 1}
            <br>
            Step: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['step'].values[0] + 1}
            <br>
            Demand: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['demand'].values[0]}
            <br>
            Load: {bus_path_df[bus_path_df["stop_id"] == stop.stop_id]['step_demand'].values[0]}
            <br>
            Has split demand?: {stop.is_split}
            """,
            popup=f"""
            <div>
                <h4>{stop.stop_name} ({stop.stop_id})</h4>
                <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
            </div>
            """,
        )

        if stop.stop_id == "0" or stop.is_split:
            c.add_to(route_map)
        else:
            c.add_to(mc)

    for stop in stops_in_disaster_area.itertuples():
        folium.CircleMarker(
            location=[stop.geometry.coords[0][1], stop.geometry.coords[0][0]],
            radius=5,
            color="red",
            fill=True,
            fill_opacity=1,
            fill_color="red",
            tooltip=f"""
            <b>{stop.stop_name} ({stop.stop_id})</b>
            """,
            popup=f"""
            <div>
                <h4>{stop.stop_name} ({stop.stop_id})</h4>
                <h4>Distance from depot: {distance_matrix.loc["0", stop.stop_id]:.1f} km</h4>
            </div>
            """,
        ).add_to(route_map)

    for route in routes_gdf.itertuples():
        route_layer = folium.FeatureGroup(f"Route {route.bus}")
        folium.PolyLine(
            locations=[(p[1], p[0]) for p in route.geometry.coords],
            color=colormap_route[route.bus],
            weight=3,
            opacity=0.5,
            tooltip=f"Route {route.bus}",
            popup=f"""
            <div>
                <h5>Route {route.bus}</h5>
                <h5>Total demand: {route.demand}</h5>
            </div>
            """,
        ).add_to(route_layer)

        route_layer.add_to(route_map)

    folium.plugins.Fullscreen(position="topright").add_to(route_map)
    folium.plugins.MousePosition(position="topright").add_to(route_map)
    folium.LayerControl().add_to(route_map)

    route_map.save(f"route_map_split_{split_type}.html")
    return route_map

# MILP model as a function

In [22]:
def solve_model(
    depot,
    stops_df,
    distance_matrix,
    num_buses,
    BUS_CAPACITY,
    DEMAND_LIMIT: int = 100,
    DISTANCE_THRESHOLD: float = 5.0,
    split_type: str = "geometric",
    MIPGap: float = 0.2,
    TimeLimit: int = 60 * 3,
    MIPFocus: int = 1,
    LogToConsole: int = 1,
):
    rng = np.random.default_rng(5)

    distance_matrix_model = distance_matrix.drop(
        columns=stops_in_disaster_area.stop_id, index=stops_in_disaster_area.stop_id
    )

    stops = list(distance_matrix_model.columns)
    num_stops = len(stops)

    distance_matrix_model = distance_matrix_model.loc[stops, stops]

    demand = {stop: rng.integers(1, DEMAND_LIMIT) for stop in stops}
    demand[stops[0]] = 0

    print('-'*100)
    print(f"Total demand: {sum(demand.values())}")
    print(f"Total capacity: {num_buses * BUS_CAPACITY}")
    print('-'*100)

    new_demand, nodes_exceeding_demand = reconstruct_demand(
        demand, BUS_CAPACITY, rng, split_type
    )

    distance_matrix_model_pruned = update_distance_matrix(
        distance_matrix_model, nodes_exceeding_demand, new_demand
    )

    for k, v in new_demand.items():
        assert v <= BUS_CAPACITY

    assert sum(new_demand.values()) == sum(demand.values())

    new_stops = list(distance_matrix_model_pruned.columns)
    num_new_stops = len(new_stops)

    print('-'*100)
    print(f"Total number of stops: {num_stops}")
    print(f"Total number of new stops (including splits): {num_new_stops}")
    print('-'*100)

    model = define_model(
        new_stops,
        num_buses,
        new_demand,
        distance_matrix_model_pruned,
        BUS_CAPACITY,
        DISTANCE_THRESHOLD=DISTANCE_THRESHOLD,
        MIPGap=MIPGap,
        TimeLimit=TimeLimit,
        MIPFocus=MIPFocus,
        LogToConsole=LogToConsole,
    )

    print('-'*100)
    print(f'Solving model with split type "{split_type.capitalize()}"')
    print('-'*100)

    model.optimize()

    distance_bus, bus_path_df, paths = show_solution(
        model,
        new_stops,
        num_buses,
        new_demand,
        nodes_exceeding_demand,
        distance_matrix_model_pruned,
        stops_df,
    )

    network_plots = plot_networkx_bus(bus_path_df)

    routes_gdf = build_route_df(bus_path_df, depot)

    route_map = plot_routes(
        distance_matrix, bus_path_df, routes_gdf, num_buses, split_type
    )

    return {
        "split_type": split_type,
        "demand": demand,
        "new_demand": new_demand,
        "model": model,
        "distance_bus": distance_bus,
        "bus_path_df": bus_path_df,
        "paths": paths,
        "network_plots": network_plots,
        "routes_gdf": routes_gdf,
        "route_map": route_map,
    }

### View solution

In [23]:
def view_model(solution):
    # Show demand and new demand splits
    demand_df = pd.DataFrame(
        solution["new_demand"].items(), columns=["stop", "new_demand"]
    )
    demand_df["original_stop"] = demand_df["stop"].apply(lambda x: x.split("_")[0])
    demand_df["original_demand"] = demand_df["original_stop"].map(solution["demand"])

    print('-'*100)
    print("Demand splits")
    print('-'*100)
    display(demand_df)

    # Show distance by bus
    print('-'*100)
    print("Distance by bus")
    print('-'*100)
    display(solution["distance_bus"])

    # Show bus paths
    print('-'*100)
    print("Bus paths")
    print('-'*100)
    display(solution["bus_path_df"])


    # Show route
    print('-'*100)
    print("Routes")
    print('-'*100)
    display(solution["route_map"])

# Run this model

In [24]:
NUM_BUSES = 15
BUS_CAPACITY = 80
DEMAND_LIMIT = 120
DISTANCE_THRESHOLD = 5.0
TIME_LIMIT = 60 * 3
LOG_TO_CONSOLE = 1
MIPGap = 0.1

## Geometric split

In [25]:
geometric_split_model = solve_model(
    depot,
    random_stops_df_gpd,
    distance_matrix,
    num_buses=NUM_BUSES,
    BUS_CAPACITY=BUS_CAPACITY,
    DEMAND_LIMIT=DEMAND_LIMIT,
    DISTANCE_THRESHOLD=DISTANCE_THRESHOLD,
    split_type="geometric",
    TimeLimit=TIME_LIMIT,
    LogToConsole=LOG_TO_CONSOLE,
    MIPGap=MIPGap,
)

----------------------------------------------------------------------------------------------------
Total demand: 934
Total capacity: 1200
----------------------------------------------------------------------------------------------------
New nodes added for: ['52718', '51511', '51983', '52142']
----------------------------------------------------------------------------------------------------
Total number of stops: 21
Total number of new stops (including splits): 41
----------------------------------------------------------------------------------------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-30
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 180
Set parameter MIPFocus to value 1
----------------------------------------------------------------------------------------------------
Solving model with split type "Geometric"
-------------------------------------------------------------------------------------

In [26]:
view_model(geometric_split_model)

----------------------------------------------------------------------------------------------------
Demand splits
----------------------------------------------------------------------------------------------------


,stop,new_demand,original_stop,original_demand
0,0,0.0,0,0
1,54638,3.0,54638,3
2,53677,56.0,53677,56
3,56274,62.0,56274,62
4,50110,75.0,50110,75
5,50828,35.0,50828,35
6,52895,7.0,52895,7
7,50716,34.0,50716,34
8,52075,46.0,52075,46
9,51388,68.0,51388,68


----------------------------------------------------------------------------------------------------
Distance by bus
----------------------------------------------------------------------------------------------------


,bus,distance
0,0,8.506836
1,1,11.155834
2,2,19.286650
3,3,7.750000
4,4,7.146941
5,5,10.938502
6,6,18.994125
7,7,6.983227
8,8,15.130114
9,9,19.538197


----------------------------------------------------------------------------------------------------
Bus paths
----------------------------------------------------------------------------------------------------


,start_stop,end_stop,distance,bus,step,demand,stop_id,stop_name,geometry,step_demand,is_split
0,0,53677,4.253418,0,0,56.0,53677,Queen-Mary / du Frère-André (Oratoire St-Joseph),POINT (-73.620431 45.493304),56.0,False
1,53677,0,4.253418,0,1,0.0,0,Depot,POINT (-73.580091 45.509642),56.0,False
2,0,56274,5.577917,1,0,62.0,56274,Sherbrooke / Metcalfe,POINT (-73.597592 45.483937),62.0,False
3,56274,0,5.577917,1,1,0.0,0,Depot,POINT (-73.580091 45.509642),62.0,False
4,0,50110,9.643325,2,0,75.0,50110,Gare Montréal-Ouest (Elmhurst / Sherbrooke),POINT (-73.641481 45.454203),75.0,False
5,50110,0,9.643325,2,1,0.0,0,Depot,POINT (-73.580091 45.509642),75.0,False
6,0,52142,3.875000,3,0,1.0,52142,Papineau / Saint-Joseph,POINT (-73.57928 45.535168),1.0,True
7,52142,52142,0.000000,3,1,14.0,52142,Papineau / Saint-Joseph,POINT (-73.57928 45.535168),15.0,True
8,52142,52142,0.000000,3,2,65.0,52142,Papineau / Saint-Joseph,POINT (-73.57928 45.535168),80.0,True
9,52142,0,3.875000,3,3,0.0,0,Depot,POINT (-73.580091 45.509642),80.0,False


----------------------------------------------------------------------------------------------------
Routes
----------------------------------------------------------------------------------------------------


## Capacity split

In [27]:
capacity_split_model = solve_model(
    depot,
    random_stops_df_gpd,
    distance_matrix,
    num_buses=NUM_BUSES,
    BUS_CAPACITY=BUS_CAPACITY,
    DEMAND_LIMIT=DEMAND_LIMIT,
    DISTANCE_THRESHOLD=DISTANCE_THRESHOLD,
    split_type="capacity",
    TimeLimit=TIME_LIMIT,
    LogToConsole=LOG_TO_CONSOLE,
    MIPGap=MIPGap,
)

----------------------------------------------------------------------------------------------------
Total demand: 934
Total capacity: 1200
----------------------------------------------------------------------------------------------------
New nodes added for: ['52718', '51511', '51983', '52142']
----------------------------------------------------------------------------------------------------
Total number of stops: 21
Total number of new stops (including splits): 25
----------------------------------------------------------------------------------------------------
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 180
Set parameter MIPFocus to value 1
----------------------------------------------------------------------------------------------------
Solving model with split type "Capacity"
----------------------------------------------------------------------------------------------------
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R

In [28]:
view_model(capacity_split_model)

----------------------------------------------------------------------------------------------------
Demand splits
----------------------------------------------------------------------------------------------------


,stop,new_demand,original_stop,original_demand
0,0,0,0,0
1,54638,3,54638,3
2,53677,56,53677,56
3,56274,62,56274,62
4,50110,75,50110,75
5,50828,35,50828,35
6,52895,7,52895,7
7,50716,34,50716,34
8,52075,46,52075,46
9,51388,68,51388,68


----------------------------------------------------------------------------------------------------
Distance by bus
----------------------------------------------------------------------------------------------------


,bus,distance
0,0,8.506836
1,1,11.155834
2,2,19.286650
3,3,5.908062
4,4,8.552004
5,5,14.067172
6,6,7.750000
7,7,24.613964
8,8,9.589808
9,10,26.820414


----------------------------------------------------------------------------------------------------
Bus paths
----------------------------------------------------------------------------------------------------


,start_stop,end_stop,distance,bus,step,demand,stop_id,stop_name,geometry,step_demand,is_split
0,0,53677,4.253418,0,0,56,53677,Queen-Mary / du Frère-André (Oratoire St-Joseph),POINT (-73.620431 45.493304),56,False
1,53677,0,4.253418,0,1,0,0,Depot,POINT (-73.580091 45.509642),56,False
2,0,56274,5.577917,1,0,62,56274,Sherbrooke / Metcalfe,POINT (-73.597592 45.483937),62,False
3,56274,0,5.577917,1,1,0,0,Depot,POINT (-73.580091 45.509642),62,False
4,0,50110,9.643325,2,0,75,50110,Gare Montréal-Ouest (Elmhurst / Sherbrooke),POINT (-73.641481 45.454203),75,False
5,50110,0,9.643325,2,1,0,0,Depot,POINT (-73.580091 45.509642),75,False
6,0,52718,2.954031,3,0,80,52718,De Lorimier / Sherbrooke,POINT (-73.56279 45.530713),80,True
7,52718,0,2.954031,3,1,0,0,Depot,POINT (-73.580091 45.509642),80,False
8,0,51511,4.276002,4,0,80,51511,Beaubien / Saint-Denis,POINT (-73.605056 45.533853),80,True
9,51511,0,4.276002,4,1,0,0,Depot,POINT (-73.580091 45.509642),80,False


----------------------------------------------------------------------------------------------------
Routes
----------------------------------------------------------------------------------------------------


## Random split

In [29]:
random_split_model = solve_model(
    depot,  
    random_stops_df_gpd,
    distance_matrix,
    num_buses=NUM_BUSES,
    BUS_CAPACITY=BUS_CAPACITY,
    DEMAND_LIMIT=DEMAND_LIMIT,
    DISTANCE_THRESHOLD=DISTANCE_THRESHOLD,
    split_type="random",
    TimeLimit=TIME_LIMIT,
    LogToConsole=LOG_TO_CONSOLE,
    MIPGap=MIPGap,
)

----------------------------------------------------------------------------------------------------
Total demand: 934
Total capacity: 1200
----------------------------------------------------------------------------------------------------
New nodes added for: ['52718', '51511', '51983', '52142']
----------------------------------------------------------------------------------------------------
Total number of stops: 21
Total number of new stops (including splits): 28
----------------------------------------------------------------------------------------------------
Set parameter MIPGap to value 0.1
Set parameter TimeLimit to value 180
Set parameter MIPFocus to value 1
----------------------------------------------------------------------------------------------------
Solving model with split type "Random"
----------------------------------------------------------------------------------------------------
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) 

In [30]:
view_model(random_split_model)

----------------------------------------------------------------------------------------------------
Demand splits
----------------------------------------------------------------------------------------------------


,stop,new_demand,original_stop,original_demand
0,0,0,0,0
1,54638,3,54638,3
2,53677,56,53677,56
3,56274,62,56274,62
4,50110,75,50110,75
5,50828,35,50828,35
6,52895,7,52895,7
7,50716,34,50716,34
8,52075,46,52075,46
9,51388,68,51388,68


----------------------------------------------------------------------------------------------------
Distance by bus
----------------------------------------------------------------------------------------------------


,bus,distance
0,0,8.506836
1,1,11.155834
2,2,19.286650
3,3,14.067172
4,4,7.750000
5,5,7.031837
6,6,6.983220
7,7,15.303080
8,8,18.994176
9,9,19.538232


----------------------------------------------------------------------------------------------------
Bus paths
----------------------------------------------------------------------------------------------------


,start_stop,end_stop,distance,bus,step,demand,stop_id,stop_name,geometry,step_demand,is_split
0,0,53677,4.253418,0,0,56,53677,Queen-Mary / du Frère-André (Oratoire St-Joseph),POINT (-73.620431 45.493304),56,False
1,53677,0,4.253418,0,1,0,0,Depot,POINT (-73.580091 45.509642),56,False
2,0,56274,5.577917,1,0,62,56274,Sherbrooke / Metcalfe,POINT (-73.597592 45.483937),62,False
3,56274,0,5.577917,1,1,0,0,Depot,POINT (-73.580091 45.509642),62,False
4,0,50110,9.643325,2,0,75,50110,Gare Montréal-Ouest (Elmhurst / Sherbrooke),POINT (-73.641481 45.454203),75,False
5,50110,0,9.643325,2,1,0,0,Depot,POINT (-73.580091 45.509642),75,False
6,0,51983,7.033586,3,0,77,51983,Beaubien / 13e Avenue,POINT (-73.585135 45.556255),77,True
7,51983,0,7.033586,3,1,0,0,Depot,POINT (-73.580091 45.509642),77,False
8,0,52142,3.875000,4,0,55,52142,Papineau / Saint-Joseph,POINT (-73.57928 45.535168),55,True
9,52142,0,3.875000,4,1,0,0,Depot,POINT (-73.580091 45.509642),55,False


----------------------------------------------------------------------------------------------------
Routes
----------------------------------------------------------------------------------------------------


# Compare models

In [31]:
models = [geometric_split_model, capacity_split_model, random_split_model]

model_bus_routes = []
for model_name, model in zip(['geometric', 'capacity', 'random'], models):
    bus_routes_df = model["bus_path_df"].copy()
    bus_routes_df["MODEL_TYPE"] = model_name
    model_bus_routes.append(bus_routes_df)

model_bus_routes_df = pd.concat(model_bus_routes)

model_bus_routes_df

,start_stop,end_stop,distance,bus,step,demand,stop_id,stop_name,geometry,step_demand,is_split,MODEL_TYPE
0,0,53677,4.253418,0,0,56.0,53677,Queen-Mary / du Frère-André (Oratoire St-Joseph),POINT (-73.620431 45.493304),56.0,False,geometric
1,53677,0,4.253418,0,1,0.0,0,Depot,POINT (-73.580091 45.509642),56.0,False,geometric
2,0,56274,5.577917,1,0,62.0,56274,Sherbrooke / Metcalfe,POINT (-73.597592 45.483937),62.0,False,geometric
3,56274,0,5.577917,1,1,0.0,0,Depot,POINT (-73.580091 45.509642),62.0,False,geometric
4,0,50110,9.643325,2,0,75.0,50110,Gare Montréal-Ouest (Elmhurst / Sherbrooke),POINT (-73.641481 45.454203),75.0,False,geometric
...,...,...,...,...,...,...,...,...,...,...,...,...
36,54638,54618,0.484414,13,4,6.0,54618,Beaubien / des Galeries-d'Anjou,POINT (-73.556338 45.594933),51.0,False,random
37,54618,51983,4.865321,13,5,25.0,51983,Beaubien / 13e Avenue,POINT (-73.585135 45.556255),76.0,True,random
38,51983,0,7.033586,13,6,0.0,0,Depot,POINT (-73.580091 45.509642),76.0,False,random
39,0,52718,2.954031,14,0,44.0,52718,De Lorimier / Sherbrooke,POINT (-73.56279 45.530713),44.0,True,random


In [32]:
# Compare distance travelled, number of buses used, number of split nodes by split_type
model_bus_routes_summary = model_bus_routes_df.groupby("MODEL_TYPE").agg(
    {
        "distance": "sum",
        "bus": "nunique",
        "stop_id": lambda x: len(x.unique()),
        "is_split": "sum",
    }
).rename(
    columns={
        "distance": "Total distance (km)",
        "bus": "Number of buses",
        "stop_id": "Number of stops",
        "is_split": "Number of split nodes",
    }
)

# Add number of 1-stop trips
model_bus_routes_summary = model_bus_routes_summary.merge(
    (
        model_bus_routes_df.groupby(["MODEL_TYPE", "bus"])
        .agg({"step": "max"})
        .query("step == 1")
        .groupby("MODEL_TYPE")
        .size()
        .to_frame()
        .rename(columns={0: "1-stop trips"})
    ),
    left_index=True,
    right_index=True,
)

model_bus_routes_summary.style.background_gradient(
    cmap="GnBu", axis=0
)

,Total distance (km),Number of buses,Number of stops,Number of split nodes,1-stop trips
MODEL_TYPE,,,,,
capacity,181.766372,13,21,8,8
geometric,141.194466,11,17,24,4
random,179.207158,14,21,11,8
